## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import math
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,59.81,81,35,11.90,scattered clouds
1,1,Hobart,AU,-42.8794,147.3294,72.27,70,75,1.99,broken clouds
2,2,Chuy,UY,-33.6971,-53.4616,63.88,79,0,4.76,clear sky
3,3,Bengkulu,ID,-3.8004,102.2655,82.81,68,100,2.46,moderate rain
4,4,Busselton,AU,-33.6500,115.3333,96.04,42,96,6.22,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Please enter the  minimum temperature (Fahrenheit) you would like "))
max_temp = float(input("Please enter the maximum temperature (Fahrenheit) you would like "))

Please enter the  minimum temperature (Fahrenheit) you would like 50
Please enter the maximum temperature (Fahrenheit) you would like 60


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,59.81,81,35,11.90,scattered clouds
15,15,Ponta Delgada,PT,37.7333,-25.6667,58.50,77,40,8.05,scattered clouds
17,17,Punta Arenas,CL,-53.1500,-70.9167,59.11,44,0,6.91,clear sky
39,39,Castro,BR,-24.7911,-50.0119,59.05,99,100,8.70,overcast clouds
44,44,Bilma,NE,18.6853,12.9164,53.35,23,0,11.03,clear sky
74,74,Ushuaia,AR,-54.8000,-68.3000,51.46,82,40,5.75,scattered clouds
99,99,Lebu,CL,-37.6167,-73.6500,57.70,92,4,4.72,clear sky
104,104,Lompoc,US,34.6391,-120.4579,58.93,62,0,0.00,clear sky
159,159,Joshimath,IN,30.5667,79.5667,51.55,35,0,0.87,clear sky
164,164,Bredasdorp,ZA,-34.5322,20.0403,59.29,75,52,6.40,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                60
City                   60
Country                60
Lat                    60
Lng                    60
Max Temp               60
Humidity               60
Cloudiness             60
Wind Speed             60
Current Description    60
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hermanus,ZA,59.81,scattered clouds,-34.4187,19.2345,
15,Ponta Delgada,PT,58.50,scattered clouds,37.7333,-25.6667,
17,Punta Arenas,CL,59.11,clear sky,-53.1500,-70.9167,
39,Castro,BR,59.05,overcast clouds,-24.7911,-50.0119,
44,Bilma,NE,53.35,clear sky,18.6853,12.9164,
74,Ushuaia,AR,51.46,scattered clouds,-54.8000,-68.3000,
99,Lebu,CL,57.70,clear sky,-37.6167,-73.6500,
104,Lompoc,US,58.93,clear sky,34.6391,-120.4579,
159,Joshimath,IN,51.55,clear sky,30.5667,79.5667,
164,Bredasdorp,ZA,59.29,broken clouds,-34.5322,20.0403,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)        
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hermanus,ZA,59.81,scattered clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
15,Ponta Delgada,PT,58.50,scattered clouds,37.7333,-25.6667,Vila Nova Hotel
17,Punta Arenas,CL,59.11,clear sky,-53.1500,-70.9167,Hotel Hain
39,Castro,BR,59.05,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
44,Bilma,NE,53.35,clear sky,18.6853,12.9164,Expeditions Ténére voyages
74,Ushuaia,AR,51.46,scattered clouds,-54.8000,-68.3000,Albatross Hotel
99,Lebu,CL,57.70,clear sky,-37.6167,-73.6500,Hostal Las Lilas
104,Lompoc,US,58.93,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
159,Joshimath,IN,51.55,clear sky,30.5667,79.5667,Shivalik Camps & Resort
164,Bredasdorp,ZA,59.29,broken clouds,-34.5322,20.0403,Bredasdorp Country Manor


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace = True)
hotel_df.dropna(subset = ['Hotel Name'], inplace = True)
clean_hotel_df = hotel_df
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hermanus,ZA,59.81,scattered clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
15,Ponta Delgada,PT,58.50,scattered clouds,37.7333,-25.6667,Vila Nova Hotel
17,Punta Arenas,CL,59.11,clear sky,-53.1500,-70.9167,Hotel Hain
39,Castro,BR,59.05,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
44,Bilma,NE,53.35,clear sky,18.6853,12.9164,Expeditions Ténére voyages
74,Ushuaia,AR,51.46,scattered clouds,-54.8000,-68.3000,Albatross Hotel
99,Lebu,CL,57.70,clear sky,-37.6167,-73.6500,Hostal Las Lilas
104,Lompoc,US,58.93,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
159,Joshimath,IN,51.55,clear sky,30.5667,79.5667,Shivalik Camps & Resort
164,Bredasdorp,ZA,59.29,broken clouds,-34.5322,20.0403,Bredasdorp Country Manor


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Can't get the figure to display

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))